In [ ]:
import sqlite3
import hashlib
from cryptography.fernet import Fernet
import tkinter as tk
from tkinter import messagebox

#Database connection and setup

def create_connection(db_file):
    """Create a database connection to the SQLite database specified by db_file."""
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Connected to {db_file} successfully.")
    except sqlite3.Error as e:
        print(e)
    return conn

def execute_sql_script(conn, script_file):
    """Execute an SQL script from a file."""
    try:
        with open(script_file, 'r') as file:
            sql_script = file.read()
        cursor = conn.cursor()
        cursor.executescript(sql_script)
        print("SQL script executed successfully.")
    except sqlite3.Error as e:
        print(e)

#Connect to the database and execute the SQL script

database = "password_manager.db"
sql_script = "database_setup.sql"
conn = create_connection(database)
if conn:
    execute_sql_script(conn, sql_script)
    conn.close()

#User registration and authentication

def hash_password(password):
    """Hash a password using SHA-256."""
    return hashlib.sha256(password.encode()).hexdigest()

def register_user(conn, username, master_password):
    """Register a new user with a hashed master password."""
    hashed_password = hash_password(master_password)
    try:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO users (username, master_password) VALUES (?, ?)", (username, hashed_password))
        conn.commit()
        print("User registered successfully.")
    except sqlite3.Error as e:
        print(e)

def authenticate_user(conn, username, master_password):
    """Authenticate a user by verifying the master password."""
    hashed_password = hash_password(master_password)
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM users WHERE username = ? AND master_password = ?", (username, hashed_password))
        user = cursor.fetchone()
        if user:
            print("User authenticated successfully.")
            return True
        else:
            print("Authentication failed.")
            return False
    except sqlite3.Error as e:
        print(e)
        return False

#Password encryption and decryption

def generate_key(master_password):
    """Generate an encryption key from the master password."""
    return hashlib.sha256(master_password.encode()).digest()

def encrypt_password(key, password):
    """Encrypt a password using the provided key."""
    fernet = Fernet(key)
    return fernet.encrypt(password.encode()).decode()

def decrypt_password(key, encrypted_password):
    """Decrypt an encrypted password using the provided key."""
    fernet = Fernet(key)
    return fernet.decrypt(encrypted_password.encode()).decode()

#Storing and retrieving passwords

def store_password(conn, user_id, website, username, password, master_password):
    """Store an encrypted password for a website."""
    key = generate_key(master_password)
    encrypted_password = encrypt_password(key, password)
    try:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO passwords (user_id, website, username, password) VALUES (?, ?, ?, ?)", (user_id, website, username, encrypted_password))
        conn.commit()
        print("Password stored successfully.")
    except sqlite3.Error as e:
        print(e)

def retrieve_passwords(conn, user_id, master_password):
    """Retrieve and decrypt all passwords for a user."""
    key = generate_key(master_password)
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT website, username, password FROM passwords WHERE user_id = ?", (user_id,))
        rows = cursor.fetchall()
        for row in rows:
            website, username, encrypted_password = row
            password = decrypt_password(key, encrypted_password)
            print(f"Website: {website}, Username: {username}, Password: {password}")
    except sqlite3.Error as e:
        print(e)

#User interface

def register():
    username = entry_username.get()
    master_password = entry_master_password.get()
    conn = create_connection(database)
    if conn:
        register_user(conn, username, master_password)
        conn.close()

def login():
    username = entry_username.get()
    master_password = entry_master_password.get()
    conn = create_connection(database)
    if conn:
        if authenticate_user(conn, username, master_password):
            messagebox.showinfo("Login", "Login successful!")
            open_password_manager(username, master_password)
        else:
            messagebox.showerror("Login", "Login failed!")
        conn.close()

def open_password_manager(username, master_password):
    password_manager_window = tk.Toplevel(root)
    password_manager_window.title("Password Manager")

def add_password():
    website = entry_website.get()
    username_pm = entry_username_pm.get()
    password = entry_password.get()
    conn = create_connection(database)
    if conn:
        cursor = conn.cursor()
        cursor.execute("SELECT id FROM users WHERE username = ?", (username,))
        user_id = cursor.fetchone()[0]
        store_password(conn, user_id, website, username_pm, password, master_password)
        conn.close()

def view_passwords():
    conn = create_connection(database)
    if conn:
        cursor = conn.cursor()
        cursor.execute("SELECT id FROM users WHERE username = ?", (username,))
        user_id = cursor.fetchone()[0]
        retrieve_passwords(conn, user_id, master_password)
        conn.close()

label_website = tk.Label(password_manager_window, text="Website")
label_website.grid(row=0, column=0)
entry_website = tk.Entry(password_manager_window)
entry_website.grid(row=0, column=1)

label_username_pm = tk.Label(password_manager_window, text="Username")
label_username_pm.grid(row=1, column=0)
entry_username_pm = tk.Entry(password_manager_window)
entry_username_pm.grid(row=1, column=1)

label_password = tk.Label(password_manager_window, text="Password")
label_password.grid(row=2, column=0)
entry_password = tk.Entry(password_manager_window, show="*")
entry_password.grid(row=2, column=1)

button_add_password = tk.Button(password_manager_window, text="Add Password", command=add_password)
button_add_password.grid(row=3, column=0, columnspan=2)

button_view_passwords = tk.Button(password_manager_window, text="View Passwords", command=view_passwords)
button_view_passwords.grid(row=4, column=0, columnspan=2)

#Main window

root = tk.Tk()
root.title("Password Manager")

label_username = tk.Label(root, text="Username")
label_username.grid(row=0, column=0)
entry_username = tk.Entry(root)
entry_username.grid(row=0, column=1)

label_master_password = tk.Label(root, text="Master Password")
label_master_password.grid(row=1, column=0)
entry_master_password = tk.Entry(root, show="*")
entry_master_password.grid(row=1, column=1)

button_register = tk.Button(root, text="Register", command=register)
button_register.grid(row=2, column=0)

button_login = tk.Button(root, text="Login", command=login)
button_login.grid(row=2, column=1)

root.mainloop()